# CSE 627 Group Project

## Configuration

In [ ]:
# Jupyter config
%load_ext rpy2.ipython
%matplotlib inline
%config InlineBackend.figure_format = 'svg'  # Or 'retina'

In [ ]:
# Python imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import *
from sklearn.mixture import *

#plt.style.use('seaborn-whitegrid')  # Set the aesthetic style of the plots

## Data Import & Pre-processing

In [ ]:
training_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

Based on <https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/>

In [ ]:
name_regex = r"""^(?P<LastName>[A-Za-z '-]+?), ((the )?(?P<Title>\w+)( of)?\.)?( (?P<FirstName>[A-Za-z'-]+?|[A-Za-z'-]+?\/[A-Za-z'-]+?))?( (?P<MiddleNames>[A-Za-z- ]+?))?( ".+| \(.+)?$"""

parsed_names = training_data['Name'].str.extract(name_regex)
training_data = training_data.assign(
    NameTitle = parsed_names['Title'],
    FirstName = parsed_names['FirstName'],
    MiddleNames = parsed_names['MiddleNames'],
    LastName = parsed_names['LastName'],
)

parsed_names = test_data['Name'].str.extract(name_regex)
test_data = test_data.assign(
    NameTitle = parsed_names['Title'],
    FirstName = parsed_names['FirstName'],
    MiddleNames = parsed_names['MiddleNames'],
    LastName = parsed_names['LastName'],
)

In [ ]:
deck_regex = r"""^(?P<Deck>[A-Z])\d+"""

parsed_decks = training_data['Cabin'].str.extract(deck_regex)
training_data = training_data.assign(
    Deck = parsed_decks['Deck'],
)

parsed_decks = test_data['Cabin'].str.extract(deck_regex)
test_data = test_data.assign(
    Deck = parsed_decks['Deck'],
)

In [ ]:
training_data = training_data.assign(
    Family_Size = training_data['SibSp'] + training_data['Parch'],
)
training_data = training_data.assign(
    Fare_Per_Person = training_data['Fare'] / (training_data['Family_Size'] + 1),
)


test_data = test_data.assign(
    Family_Size = test_data['SibSp'] + test_data['Parch'],
)
test_data = test_data.assign(
    Fare_Per_Person = test_data['Fare'] / (test_data['Family_Size'] + 1),
)

In [ ]:
training_data

In [ ]:
training_data.to_csv('train_processed.csv')
test_data.to_csv('test_processed.csv')